In [ ]:
import numpy as np
import scipy.signal as ss

import matplotlib
import matplotlib.pyplot as plt

spikes = np.load('/home/e/etayhay/frankm/Mazza2023/data/simulations_raw/Healthy/SPIKES_Seed1000.npy',allow_pickle=True).item(0)
spikecount = np.load('/home/e/etayhay/frankm/Mazza2023/data/figures/Figure5/spikecount.npy')
phase = np.load('/home/e/etayhay/frankm/Mazza2023/data/figures/Figure5/phase.npy')
eeg_low = np.load('/home/e/etayhay/frankm/Mazza2023/data/figures/Figure5/eeg_low.npy')
eeg = np.load('/home/e/etayhay/frankm/Mazza2023/data/figures/Figure5/eeg.npy')
spikehists = np.load('/home/e/etayhay/frankm/Mazza2023/data/figures/Figure5/spikehists.npy')
radbins = np.load('/home/e/etayhay/frankm/Mazza2023/data/figures/Figure5/radbins.npy')


def myround(x, prec=4, base=.025):
    return round(base * round(float(x)/base),prec)


In [ ]:
fig = plt.figure(figsize = (12,8))
xmin = 13500
xmax = 14500
hclr = 'k'
mddclr = 'red'
colors = ['k','red','green','orange']

font = {'family' : 'arial','weight' : 'normal','size'   : 10}
labelfont = {'family' : 'arial','weight' : 'bold','size'   : 14}
popnames = ['HL23PN1', 'HL23MN1', 'HL23BN1', 'HL23VN1']
pop_colors = {'HL23PN1':'k', 'HL23MN1':'red', 'HL23BN1':'green', 'HL23VN1':'orange'}

matplotlib.rc('font', **font)
matplotlib.rc('legend',**{'fontsize':10})

y = .95
x = -.22

rstart =  0
rwidth = .3

vshift =.07



tvec_round = np.asarray([myround(i) for i in np.arange(0,30000.025,0.025)])

h_raster = fig.add_axes([rstart,.6-vshift,rwidth,.4])
for name, spts, gids in zip(popnames, spikes['times'], spikes['gids']):
    t = []
    g = []
    for spt, gid in zip(spts, gids):
        t = np.r_[t, spt]
        g = np.r_[g, np.zeros(spt.size)+gid]
    h_raster.plot(t[t >= xmin], g[t >= xmin], 's', color=pop_colors[name], markersize=1, marker='o')

h_raster.set_ylabel('Cell Number',**font)
h_raster.set_xticklabels(())
h_raster.set_ylim(0,1000)
h_raster.set_xlim(xmin,xmax)

PN_h_sr = fig.add_axes([rstart,.41-vshift,rwidth,.15])
PNr = spikecount

ratesx = np.arange(200,3000)
PN_h_sr.plot(ratesx,PNr,color='k')
PN_h_sr.set_xlim(xmin/10,xmax/10)
PN_h_sr.set_xticklabels([])
PN_h_sr.set_ylim(0,25)
PN_h_sr.set_ylabel('Spike Count',**font)
PN_h_sr.set_title('B',loc='left',y=y,x=x,**labelfont)
PN_h_sr.spines['right'].set_visible(False)
PN_h_sr.spines['top'].set_visible(False)

EEG_h_p = fig.add_axes([rstart,.21,rwidth,.08])
EEG_h_p.plot(tvec_round,phase,c='green',alpha=1)
EEG_h_p.set_ylabel('Degrees',**font)
EEG_h_p.set_xticklabels(())
EEG_h_p.set_title('C',loc='left',y=y,x=x,**labelfont)
EEG_h_p.set_yticks([-180, -90, 0, 90, 180])
EEG_h_p.set_yticklabels([-180, None, 0, None, 180])

shift = abs(np.mean(eeg_low[int(40000*(xmin/1000)):int(40000*(xmax/1000))]))
EEG_h_bp = fig.add_axes([rstart,0,rwidth,.15])
EEG_h_bp.plot(tvec_round[int(40000*(xmin/1000)):int(40000*(xmax/1000))],eeg_low[int(40000*(xmin/1000)):int(40000*(xmax/1000))]+shift,c='k',alpha=1)
EEG_h_bp.plot(tvec_round[int(40000*(xmin/1000)):int(40000*(xmax/1000))],eeg[int(40000*(xmin/1000)):int(40000*(xmax/1000))]+.0000034,c='k',alpha=.3,lw=.5)
EEG_h_bp.set_ylabel('Potential (mV)',**font)
EEG_h_bp.set_xlabel('Time (ms)')
EEG_h_bp.set_title('D',loc='left',y=y,x=x,**labelfont)
# EEG_h_bp.set_ylim(-0.000002, 0.000002)
# EEG_h_bp.set_yticks([-0.000002,-0.000001,0,0.000001,0.000002])

axes = [EEG_h_bp,EEG_h_p,h_raster]
xlims = [ax.set_xlim(xmin,xmax) for ax in axes]
rspines = [ax.spines['right'].set_visible(False) for ax in axes]
tspines = [ax.spines['top'].set_visible(False) for ax in axes]


def plot_pop_sd(fig,dims,pop):
    
    ax = fig.add_axes(dims,projection='polar')
    
    colors = [['k', 'red','green','orange'],['red','red','red','red']]
    ls = ['-','--']
    for c in range(2):
        mean = spikehists[c,pop,:].mean(axis=0)
        mean = np.insert(mean,0,mean[-1])
        mean = np.append(mean,mean[-1])

        sd = spikehists[c,pop,:].std(axis=0)
        sd = np.insert(sd,0,sd[-1])
        sd = np.append(sd,sd[-1])

        sd_top = (mean+sd)/mean
        sd_bottom = (mean-sd)/mean

        mean = mean/max(mean)
        sd = sd/max(sd)

        sd_top = mean*sd_top
        sd_bottom = mean*sd_bottom


        ax.plot(radbins,mean,color=colors[c][pop],linestyle = ls[c])
        ax.fill_between(radbins,sd_top,sd_bottom,alpha=.2,color=colors[c][pop])
        ax.set_yticks([.2,.4,.6,.8,1.0])
    
    return ax

r = .38
PN = plot_pop_sd(fig,[.3,.53,r,r],0)
SST = plot_pop_sd(fig,[.65,.53,r,r],1)
PV = plot_pop_sd(fig,[.3,0,r,r],2)
VIP = plot_pop_sd(fig,[.65,0,r,r],3)

fig.text(x=-.065,y=.98,s='A',**labelfont)

fig.text(x=.33,y=.98,s='E',**labelfont)
fig.text(x=.48,y=.96,s='Pyr',fontsize=11,weight='bold')
fig.text(x=.83,y=.96,s='SST',fontsize=11,weight='bold')
fig.text(x=.48,y=.43,s='PV',fontsize=11,weight='bold')
fig.text(x=.83,y=.43,s='VIP',fontsize=11,weight='bold')

fig.savefig('/home/e/etayhay/frankm/Mazza2023/figures/figoutput/Figure5.jpeg', facecolor='white', edgecolor='none',bbox_inches = "tight",dpi=600)
